## Function Calling in OpenAI:  Finance Assistant

In [ ]:
#!pip install openai
#!pip install python-dotenv

In [1]:
import json
import pandas as pd
import openai
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("openai_api_key")

client = openai.OpenAI()

**calculateLoanPayment()** is the function we will incorporate into the tools available to our AI Financial Assistant.   
The function has three required input arguements:  principal, interestRate, term.   
The function returns the monthly payment for a loan with the specified parameters.

In [2]:
def calculateLoanPayment(principal: float, interestRate: float, term: int) -> float:
    monthly_rate = interestRate / 100 / 12
    total_payments = term * 12
    
    if monthly_rate == 0:
        monthly_payment = principal / total_payments
    else:
        monthly_payment = (principal * monthly_rate * (1 + monthly_rate) ** total_payments) / ((1 + monthly_rate) ** total_payments - 1)
    
    return monthly_payment



In [3]:
principal = 50000  
interestRate = 5
term = 15  

monthly_payment = calculateLoanPayment(principal, interestRate, term)
print(f"The monthly payment is: ${monthly_payment:.2f}")

The monthly payment is: $395.40


Create schema for function call.

In [3]:
parameters = {
    "type": "object",
    "properties": {
        "principal": {
            "type": "number",
            "description": "The loan amount or principal in dollars."
        },
        "interestRate": {
            "type": "number",
            "description": "The annual interest rate as a percentage (e.g., 5 for 5%)."
        },
        "term": {
            "type": "integer",
            "description": "The loan term in years."
        }
    },
    "required": ["principal", "interestRate", "term"]
}

calc_tool = {
    "type": "function",
    "function": {
        "name": "calculateLoanPayment",
        "description": "Calculates loan payment from user inputs",
        "parameters": parameters,
    }
}

In [5]:
def get_output(messages_input, model_input="gpt-4o-mini", temperature_input = 0) :

    response = client.chat.completions.create(
        model = model_input,
        messages = messages_input,
        temperature = temperature_input,
        tools = [calc_tool],
        tool_choice = "auto"
    )
    
    response = response.choices[0].message
    return response

In [6]:
def initialize_bot():
    system_message = f"""
    You are a financial assistant chatbot designed to help users understand their loan payment options. \
    First, greet the user.  Ask the user questions as needed to understnand their financing needs.
    Answer any questions they might have about terms. \
    Your goal is to understand the user's needs in terms of their loan amount (principal), annual interest rate (in percentage), \
    and loan term (in years). \
    Once you have all needed information, use the calculateLoanPayment function to compute the monthly loan payment.\
    Tell the user their calculated monthly payment amount in dollars. \
    If the user needs further assistance, provide insights into adjusting loan terms, rates, or principal to achieve their financial goals.
    """
    
    context = [{"role":"system", "content":system_message}]

    return context


In [8]:
def collect_messages(prompt, context):
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_output(context) 
    context.append({'role':'assistant', 'content':f"{response.content}"})
    return response

In [11]:
def main():
    context = initialize_bot()

    print("Welcome! Type 'exit' to end the chat.")
    greeting = collect_messages("", context)
    print(greeting.content)
    
    while True:
        user_input = input()
        
        if user_input.lower() == 'exit':
            return context
        
        response = collect_messages(user_input, context)

        # Display the assistant's response if it has content 
        if response.content:
            print(response.content)

        # Execute function call if assistant's response has tool call
        if response.tool_calls:
            # Extract the function arguments
            arguments = json.loads(response.tool_calls[0].function.arguments)
            principal = arguments["principal"]
            interestRate = arguments["interestRate"]
            term = arguments["term"]

            # Call the calculateLoanPayment function with extracted arguments
            monthly_payment = calculateLoanPayment(principal, interestRate, term)
            context.append({'role':'function', \
                            'name': 'calculateLoanPayment', \
                            'content':f"Calculated Monthly Payment: ${monthly_payment:.2f}"})
            
            # Add another LLM call to generate new assistant response to customer
            response = collect_messages("return monthly payment to customer", context)
            print(response.content)
    
        

In [12]:
chat_transcript = main()

Welcome! Type 'exit' to end the chat.
Hello! How can I assist you today with your loan payment options? Do you have a specific loan amount, interest rate, or term in mind?


 I need to buy a new car


That's great! Buying a new car is an exciting decision. To help you with your financing options, could you please provide me with the following details?

1. What is the loan amount (principal) you are considering for the car?
2. What is the annual interest rate (in percentage) you expect?
3. How many years do you plan to take the loan for (loan term)? 

Feel free to share any other details or questions you might have!


 The car is 25,000.  What do you think is a good interest rate?


The interest rate for a car loan can vary based on several factors, including your credit score, the lender, and the current market conditions. Generally, a good interest rate for a new car loan can range from about 3% to 6% for borrowers with good credit. 

If you have excellent credit, you might even find rates lower than 3%. If your credit is not as strong, the rate could be higher.

Do you have an idea of your credit score or the rate you might qualify for? If not, we can proceed with a common rate, like 5%, for our calculations. Would that work for you? Also, how long do you plan to finance the car (in years)?


 Let's use 5%.  I want a 10 year loan


Great! To summarize, you are looking to finance a car with the following details:

- Loan Amount (Principal): $25,000
- Annual Interest Rate: 5%
- Loan Term: 10 years

I'll calculate your monthly loan payment based on these details. One moment, please!
Your calculated monthly payment for the car loan is approximately **$265.16**. 

If you have any further questions or need assistance with adjusting the loan terms, rates, or principal to better fit your financial goals, feel free to ask!


 exit


In [13]:
chat_transcript

[{'role': 'system',
  'content': "\n    You are a financial assistant chatbot designed to help users understand their loan payment options.     First, greet the user.  Ask the user questions as needed to understnand their financing needs.\n    Answer any questions they might have about terms.     Your goal is to understand the user's needs in terms of their loan amount (principal), annual interest rate (in percentage),     and loan term (in years).     Once you have all needed information, use the calculateLoanPayment function to compute the monthly loan payment.    Tell the user their calculated monthly payment amount in dollars.     If the user needs further assistance, provide insights into adjusting loan terms, rates, or principal to achieve their financial goals.\n    "},
 {'role': 'user', 'content': ''},
 {'role': 'assistant',
  'content': 'Hello! How can I assist you today with your loan payment options? Do you have a specific loan amount, interest rate, or term in mind?'},
 {'r